In [5]:
import os 
results_path = os.path.join(os.getcwd(), "results")
paths = [os.path.join(results_path, x) for x in os.listdir(results_path)]
paths

['/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_none_dsprites_model_cnn_rep_2',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_simclr_dsprites_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/.DS_Store',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_simclr3_dsprites_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_simclr2_dsprites_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_none_shapes3d_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_none_dsprites_model_cnn_rep_1',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_sup_dsprites_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_none_dsprites_model_cnn_rep_0',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_crop_dsprites_model_cnn_rep_1',
 '/Users/miguelmartins/Projects/csi-robustness/results/diet_au

In [10]:
import re
import pandas as pd

import re
import pandas as pd
def parse_logs(log_path):
    
    with open(log_path, "r") as f:
        text = f.read()
    
    # Regexes for the lines we care about
    coord_re = re.compile(r"^Coordinate\s+(\d+)\s+(\w+)\s+([-\d\.eE+]+)")
    val_re   = re.compile(r"^val\s+Epoch\s+(\d+)\s+Loss\s+([-\d\.eE+]+)\s+val_acc\s+([-\d\.eE+]+)")
    adv_re   = re.compile(r"^adv\s+Epoch\s+(\d+)\s+Loss\s+([-\d\.eE+]+)\s+adv_acc\s+([-\d\.eE+]+)")
    
    rows_val = []
    rows_adv = []
    coords = {}   # holds shape/scale/orientation/posX/posY before each val/adv line
    
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
    
        # Coordinate lines
        m = coord_re.match(line)
        if m:
            idx, name, value = m.groups()
            coords[name] = float(value)
            continue
    
        # Validation block
        m = val_re.match(line)
        if m:
            epoch, loss, acc = m.groups()
            row = {
                "epoch": int(epoch),
                "loss": float(loss),
                "val_acc": float(acc),
            }
            row.update(coords)        # add shape/scale/orientation/posX/posY
            rows_val.append(row)
            coords = {}               # reset for next block
            continue
    
        # Adversarial block
        m = adv_re.match(line)
        if m:
            epoch, loss, acc = m.groups()
            row = {
                "epoch": int(epoch),
                "loss": float(loss),
                "adv_acc": float(acc),
            }
            row.update(coords)
            rows_adv.append(row)
            coords = {}
            continue
    
    # Build the two DataFrames
    df_val = pd.DataFrame(rows_val)
    df_adv = pd.DataFrame(rows_adv)
    
    return df_val, df_adv


In [11]:
dir_ = '/Users/miguelmartins/Projects/csi-robustness/results/diet_aug_none_dsprites_model_cnn_rep_0'
log_path = os.path.join(dir_, "log_probe.txt")   # <-- change if needed

df_val, df_adv = parse_logs(log_path)